In [0]:
corpus = ['kral bir adamdir',
'kral ile kralice evlenir',
'kralice ile kral evlenir',
'biraz gereksiz sozler',
'kralice bir kadindir']

In [0]:
import numpy as np
#np.random.seed(None)


from keras.optimizers import Adam


In [29]:
from keras.preprocessing.text import one_hot
vocs = 50

cumlem = np.array(one_hot("kral kralice kralice adamdir kadindir sozler bir ile evlenir ",vocs))
cumlem

array([30, 16, 16, 37, 14, 36, 18, 16, 46])

In [30]:
from sklearn.preprocessing import OneHotEncoder
one_hot_enc = OneHotEncoder()
one_hot_enc.fit(np.array(range(vocs)).reshape(-1,1))
oh_enc = one_hot_enc.transform(cumlem.reshape(-1,1)).toarray()
oh_enc[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [31]:

encoded_docs = [one_hot(cumle,vocs) for cumle in corpus]
encoded_docs[0]

[30, 18, 37]

In [34]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 5
padded_docs = pad_sequences(encoded_docs, maxlen = maxlen, padding="post")
padded_docs

array([[30, 18, 37,  0,  0],
       [30, 16, 16, 46,  0],
       [16, 16, 30, 46,  0],
       [15, 35, 36,  0,  0],
       [16, 18, 14,  0,  0]], dtype=int32)

In [35]:
tuples = np.empty((0,2))

for cumle in padded_docs:
  length = len(cumle)

  for i in range(length):
    if cumle[i] !=0:
      if i<length-1 & cumle[i+1] != 0:
        tuples = np.append(tuples, [[cumle[i], cumle[i+1]]], axis = 0)

        if i<length-2 & cumle[i+2] != 0:
          tuples = np.append(tuples, [[cumle[i], cumle[i+2]]], axis = 0)
      
      if i > 0 :
        tuples = np.append(tuples, [[cumle[i],cumle[i-1]]],axis = 0)

        if i>1:
          tuples = np.append(tuples, [[cumle[i],cumle[i-2]]],axis = 0)

      

print(tuples.shape)
tuples




(26, 2)


array([[18., 37.],
       [18., 30.],
       [37., 18.],
       [37., 30.],
       [16., 30.],
       [16., 46.],
       [16., 16.],
       [16., 30.],
       [46., 16.],
       [46., 16.],
       [16., 30.],
       [16., 46.],
       [16., 16.],
       [30., 46.],
       [30., 16.],
       [30., 16.],
       [46., 30.],
       [46., 16.],
       [35., 36.],
       [35., 15.],
       [36., 35.],
       [36., 15.],
       [18., 14.],
       [18., 16.],
       [14., 18.],
       [14., 16.]])

In [0]:
onehotlabels_x = one_hot_enc.transform(tuples[:,0].reshape(-1,1)).toarray()
onehotlabels_y = one_hot_enc.transform(tuples[:,1].reshape(-1,1)).toarray()

In [38]:
print(onehotlabels_y.shape)

(26, 50)


In [47]:
from keras.layers import Input
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Input


model = Sequential()

#model.add(Input(shape=(50)))

bottleneck = Dense(2,activation = 'relu', input_shape=(50,))
model.add(bottleneck)
model.add(Dense(vocs,activation="softmax"))

model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["acc"])

model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 2)                 102       
_________________________________________________________________
dense_9 (Dense)              (None, 50)                150       
Total params: 252
Trainable params: 252
Non-trainable params: 0
_________________________________________________________________


In [48]:
hist = model.fit(onehotlabels_x,onehotlabels_y,epochs=100)

Epoch 1/100
26/26 [==============================] - 0s 9ms/step - loss: 0.0980 - acc: 0.9800
Epoch 2/100
26/26 [==============================] - 0s 73us/step - loss: 0.0980 - acc: 0.9800
Epoch 3/100
26/26 [==============================] - 0s 88us/step - loss: 0.0980 - acc: 0.9800
Epoch 4/100
26/26 [==============================] - 0s 58us/step - loss: 0.0979 - acc: 0.9800
Epoch 5/100
26/26 [==============================] - 0s 61us/step - loss: 0.0979 - acc: 0.9800
Epoch 6/100
26/26 [==============================] - 0s 93us/step - loss: 0.0979 - acc: 0.9800
Epoch 7/100
26/26 [==============================] - 0s 145us/step - loss: 0.0978 - acc: 0.9800
Epoch 8/100
26/26 [==============================] - 0s 75us/step - loss: 0.0978 - acc: 0.9800
Epoch 9/100
26/26 [==============================] - 0s 78us/step - loss: 0.0978 - acc: 0.9800
Epoch 10/100
26/26 [==============================] - 0s 92us/step - loss: 0.0977 - acc: 0.9800
Epoch 11/100
26/26 [=============================

In [51]:
printoutmodel = Sequential()
printoutmodel.add(bottleneck)
printoutmodel.compile(optimizer='adam',loss="binary_crossentropy")
test = printoutmodel.predict(oh_enc)
print(test)

[[0.3755498  0.        ]
 [0.07502753 0.        ]
 [0.07502753 0.        ]
 [0.10645559 0.3991741 ]
 [0.51284546 0.06801385]
 [0.         0.        ]
 [0.0304682  0.00197288]
 [0.07502753 0.        ]
 [0.         0.        ]]
